## Nume studenti:
- Alexandra Manole
- Teodor Mihaescu

## Grupa: 382

# Data Set 2: Energy Efficency
### (Missing values: no)

In [1354]:
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV

In [1355]:
# Citeste datele
data = pd.read_csv('./Datasets/energy.csv').iloc[:768,:-2]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      768 non-null    float64
 1   X2      768 non-null    float64
 2   X3      768 non-null    float64
 3   X4      768 non-null    float64
 4   X5      768 non-null    float64
 5   X6      768 non-null    float64
 6   X7      768 non-null    float64
 7   X8      768 non-null    float64
 8   Y1      768 non-null    float64
 9   Y2      768 non-null    float64
dtypes: float64(10)
memory usage: 60.1 KB


In [1356]:
# Pregateste datele
X = data.loc[:, :'X8']
# y = data.loc[:, 'Y1':]
y1 = data.loc[:, 'Y1']
y2 = data.loc[:, 'Y2']

In [1357]:
# Transformam problema dintr-una de regresie intr-una de clasificare, dand intervale de clasificare y-ilor
"""
Observam ca y1 si y2 se incadreaza intre urmatoarele valori:
y1: 6.01-43.1
y2: 10.9-48.03

Vom imparti datele in urmatoarele clase:
y1: 6-10, 10-20, 20-30, 30-40, 40-50
y2: 10-20, 20-30, 30-40, 40-50
"""
y1 = np.where(np.logical_and(y1>=6, y1<10), 0, y1)
y1 = np.where(np.logical_and(y1>=10, y1<20), 1, y1)
y1 = np.where(np.logical_and(y1>=20, y1<30), 2, y1)
y1 = np.where(np.logical_and(y1>=30, y1<40), 3, y1)
y1 = np.where(np.logical_and(y1>=40, y1<50), 4, y1)

y2 = np.where(np.logical_and(y2>=10, y2<20), 0, y2)
y2 = np.where(np.logical_and(y2>=20, y2<30), 1, y2)
y2 = np.where(np.logical_and(y2>=30, y2<40), 2, y2)
y2 = np.where(np.logical_and(y2>=40, y2<50), 3, y2)

y = np.stack((y1, y2), axis=1)

In [1358]:
# Scaleaza datele
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)
# y = min_max_scaler.fit_transform(y)

In [1359]:
# Separarea setului de date
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, shuffle=True)

In [1360]:
# Verifica daca sunt egal distribuite
print(Counter(y_train[:,0]))
print(Counter(y_test[:,0]))
print("\n")
print(Counter(y_train[:,1]))
print(Counter(y_test[:,1]))

Counter({1.0: 247, 2.0: 117, 3.0: 115, 4.0: 23, 0.0: 10})
Counter({1.0: 130, 3.0: 52, 2.0: 52, 4.0: 12, 0.0: 10})


Counter({0.0: 227, 2.0: 148, 1.0: 112, 3.0: 25})
Counter({0.0: 121, 2.0: 68, 1.0: 53, 3.0: 14})


## Model 1: LogisticRegression

In [1361]:
# Model
"""
"""
model1 = MultiOutputClassifier(LogisticRegression(random_state=0, multi_class='multinomial'))
model1.fit(X_train, y_train)
y_hat = model1.predict(X_test)

In [1362]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[3. 2.]
 [2. 1.]
 [2. 1.]
 [3. 2.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]

Ground truth:
[[3. 2.]
 [1. 1.]
 [2. 1.]
 [3. 2.]
 [1. 0.]
 [1. 1.]
 [1. 1.]
 [1. 0.]
 [0. 0.]
 [1. 0.]]

Failed:	[46 54]


In [1363]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [1364]:
# Custom scoreers for multiclass output
f1_scorer = metrics.make_scorer(metrics.f1_score, greater_is_better=False, average='micro')
accuracy_scorer = metrics.make_scorer(metrics.accuracy_score, greater_is_better=True)

In [1365]:
# Cross Validation Y1
results_train = cross_validate(model1, X, y1,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.05884266, 0.06056428, 0.04687476, 0.04389167, 0.04288197]),
 'score_time': array([0.00498724, 0.00299144, 0.00199437, 0.00498843, 0.00099778]),
 'test_f1_scorer': array([-0.80582524, -0.7961165 , -0.87254902, -0.85294118, -0.8627451 ]),
 'train_f1_scorer': array([-0.86552567, -0.8606357 , -0.85609756, -0.83658537, -0.84878049]),
 'test_accuracy': array([0.80582524, 0.7961165 , 0.87254902, 0.85294118, 0.8627451 ]),
 'train_accuracy': array([0.86552567, 0.8606357 , 0.85609756, 0.83658537, 0.84878049])}

In [1367]:
# Cross Validation Y2
results_train = cross_validate(model1, X, y2,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.03989291, 0.04587603, 0.03192091, 0.03989148, 0.04085636]),
 'score_time': array([0.0010004 , 0.00299358, 0.00298738, 0.00399518, 0.00211287]),
 'test_f1_scorer': array([-0.77669903, -0.7961165 , -0.7745098 , -0.74509804, -0.80392157]),
 'train_f1_scorer': array([-0.78973105, -0.78239609, -0.79512195, -0.79756098, -0.78292683]),
 'test_accuracy': array([0.77669903, 0.7961165 , 0.7745098 , 0.74509804, 0.80392157]),
 'train_accuracy': array([0.78973105, 0.78239609, 0.79512195, 0.79756098, 0.78292683])}

In [1369]:
# Cautare hiperparametrii optimi - GridSearch (doar pentru Y1)
parameter_grid = {
    'C' : np.linspace(1000, 2000, 10),# C = 1/lambda ; C mare => lambda mic
    'solver': ['saga', 'lbfgs']}
grid_search = GridSearchCV(estimator = LogisticRegression(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train, y1_train)
results_gscv = cross_val_score(grid_search, X_test, y1_test, scoring='accuracy')

In [1370]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.8594268476621417
{'C': 1222.2222222222222, 'solver': 'lbfgs'}


## Model 2: KNeighborsClassifier

In [1371]:
# Model
"""
"""
model2 = KNeighborsClassifier()
model2.fit(X_train, y_train)
y_hat = model2.predict(X_test)

In [1372]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[3. 2.]
 [1. 1.]
 [2. 1.]
 [2. 2.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 0.]
 [1. 0.]]

Ground truth:
[[3. 2.]
 [1. 1.]
 [2. 1.]
 [3. 2.]
 [1. 0.]
 [1. 1.]
 [1. 1.]
 [1. 0.]
 [0. 0.]
 [1. 0.]]

Failed:	[41 53]


In [1373]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [1374]:
# Cross Validation Y1
results_train = cross_validate(model2, X, y1,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00199556, 0.0009973 , 0.00099611, 0.00236106, 0.00199556]),
 'score_time': array([0.01048613, 0.00598359, 0.00598407, 0.01200843, 0.00997233]),
 'test_f1_scorer': array([-0.80582524, -0.86407767, -0.85294118, -0.83333333, -0.85294118]),
 'train_f1_scorer': array([-0.91442543, -0.91198044, -0.91219512, -0.91707317, -0.91707317]),
 'test_accuracy': array([0.80582524, 0.86407767, 0.85294118, 0.83333333, 0.85294118]),
 'train_accuracy': array([0.91442543, 0.91198044, 0.91219512, 0.91707317, 0.91707317])}

In [1376]:
# Cross Validation Y2
results_train = cross_validate(model2, X, y2,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00099564, 0.00199509, 0.00200653, 0.0009973 , 0.00099421]),
 'score_time': array([0.01196742, 0.00901175, 0.00896549, 0.00598693, 0.00498652]),
 'test_f1_scorer': array([-0.75728155, -0.83495146, -0.7254902 , -0.74509804, -0.80392157]),
 'train_f1_scorer': array([-0.88508557, -0.85330073, -0.88536585, -0.88536585, -0.87804878]),
 'test_accuracy': array([0.75728155, 0.83495146, 0.7254902 , 0.74509804, 0.80392157]),
 'train_accuracy': array([0.88508557, 0.85330073, 0.88536585, 0.88536585, 0.87804878])}

In [1378]:
# Cautare hiperparametrii optimi - GridSearch (doar pentru Y1)
parameter_grid = {'n_neighbors': list(range(1, 20)), 'p': [1, 2, 3]}
grid_search = GridSearchCV(estimator = KNeighborsClassifier(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train, y1_train)
results_gscv = cross_val_score(grid_search, X_test, y1_test, scoring='accuracy')

In [1379]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.879185520361991
{'n_neighbors': 4, 'p': 1}


## Model 3: DecisionTreeClassifier

In [1380]:
# Model
"""
"""
model3 = DecisionTreeClassifier(max_depth=2)
model3.fit(X_train, y_train)
y_hat = model3.predict(X_test)

In [1381]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[2. 1.]
 [2. 1.]
 [2. 1.]
 [2. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 0.]
 [1. 0.]]

Ground truth:
[[3. 2.]
 [1. 1.]
 [2. 1.]
 [3. 2.]
 [1. 0.]
 [1. 1.]
 [1. 1.]
 [1. 0.]
 [0. 0.]
 [1. 0.]]

Failed:	[47 75]


In [1382]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [1383]:
# Cross Validation Y1
results_train = cross_validate(model3, X, y1,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00099826, 0.        , 0.00099683, 0.00099754, 0.0019896 ]),
 'score_time': array([0.00299001, 0.00199318, 0.0029912 , 0.00199556, 0.00199533]),
 'test_f1_scorer': array([-0.78640777, -0.80582524, -0.85294118, -0.7745098 , -0.85294118]),
 'train_f1_scorer': array([-0.82151589, -0.81662592, -0.80487805, -0.82439024, -0.80487805]),
 'test_accuracy': array([0.78640777, 0.80582524, 0.85294118, 0.7745098 , 0.85294118]),
 'train_accuracy': array([0.82151589, 0.81662592, 0.80487805, 0.82439024, 0.80487805])}

In [1385]:
# Cross Validation Y2
results_train = cross_validate(model3, X, y2,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.0009985 , 0.00099874, 0.00099754, 0.00199795, 0.00100088]),
 'score_time': array([0.0029943 , 0.0019927 , 0.00099754, 0.00299072, 0.00202799]),
 'test_f1_scorer': array([-0.76699029, -0.74757282, -0.74509804, -0.71568627, -0.7254902 ]),
 'train_f1_scorer': array([-0.73349633, -0.73838631, -0.73902439, -0.74634146, -0.74390244]),
 'test_accuracy': array([0.76699029, 0.74757282, 0.74509804, 0.71568627, 0.7254902 ]),
 'train_accuracy': array([0.73349633, 0.73838631, 0.73902439, 0.74634146, 0.74390244])}

In [1387]:
# Cautare hiperparametrii optimi - GridSearch (doar pentru Y1)
parameter_grid = {'max_depth': list(range(1,5)),
'min_samples_split': np.linspace(1,3,5),
'min_samples_leaf': np.linspace(0.1,0.5,5),
'max_features': ['sqrt', 'log2', None],
'criterion': ['gini', 'entropy']}
grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train, y1_train)
results_gscv = cross_val_score(grid_search, X_test, y1_test, scoring='accuracy')

In [1388]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.6130467571644043
{'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 1.0}


## Model 4: RandomForestClassifier

In [1389]:
# Model
"""
"""
model4 = RandomForestClassifier(max_depth=2, random_state=0, ccp_alpha=0.2)
model4.fit(X_train, y_train)
y_hat = model4.predict(X_test)

In [1390]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[2. 2.]
 [2. 2.]
 [2. 2.]
 [2. 2.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]

Ground truth:
[[3. 2.]
 [1. 1.]
 [2. 1.]
 [3. 2.]
 [1. 0.]
 [1. 1.]
 [1. 1.]
 [1. 0.]
 [0. 0.]
 [1. 0.]]

Failed:	[79 67]


In [1391]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [1392]:
# Cross Validation Y1
results_train = cross_validate(model4, X, y1,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.24839854, 0.24235201, 0.25481319, 0.23836088, 0.28424096]),
 'score_time': array([0.0150063 , 0.01197171, 0.01496053, 0.01091933, 0.01595831]),
 'test_f1_scorer': array([-0.67961165, -0.67961165, -0.70588235, -0.71568627, -0.69607843]),
 'train_f1_scorer': array([-0.6992665 , -0.6992665 , -0.69268293, -0.6902439 , -0.69512195]),
 'test_accuracy': array([0.67961165, 0.67961165, 0.70588235, 0.71568627, 0.69607843]),
 'train_accuracy': array([0.6992665 , 0.6992665 , 0.69268293, 0.6902439 , 0.69512195])}

In [1394]:
# Cross Validation Y2
results_train = cross_validate(model4, X, y2,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.26072931, 0.27127862, 0.32481742, 0.26030445, 0.25671387]),
 'score_time': array([0.01798368, 0.02991724, 0.02393413, 0.02493215, 0.01097226]),
 'test_f1_scorer': array([-0.73786408, -0.73786408, -0.73529412, -0.7254902 , -0.7254902 ]),
 'train_f1_scorer': array([-0.73105134, -0.73105134, -0.73170732, -0.73414634, -0.73414634]),
 'test_accuracy': array([0.73786408, 0.73786408, 0.73529412, 0.7254902 , 0.7254902 ]),
 'train_accuracy': array([0.73105134, 0.73105134, 0.73170732, 0.73414634, 0.73414634])}

In [1398]:
# Cautare hiperparametrii optimi - GridSearch (doar pentru Y1)
parameter_grid = {'max_depth': list(range(5,10)),
'min_samples_split': list(range(1,4)),
'min_samples_leaf': list(range(1,5))}
grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train, y1_train)
results_gscv = cross_val_score(grid_search, X_test, y1_test, scoring='accuracy')

In [1399]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.9453242835595776
{'max_depth': 8, 'min_samples_leaf': 1, 'min_samples_split': 2}


## Model 5: SVC (Support Vector Classification)

In [1347]:
# Model
"""
Folosim metoda SVC pentru a clasifica observatiile. Atunci cand datele sunt in 4 sau mai multe dimensiuni SVC-ul este un hiper-plan. Acesta poate lucra cu valori extreme si din cazua ca permite clasificari eronate poate lucra si cu clasificari suprapuse.
"""
model5 = MultiOutputClassifier(SVC(C=2, gamma='auto'))
model5.fit(X_train, y_train)
y_hat = model5.predict(X_test)

In [1348]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[3. 2.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [3. 2.]
 [1. 0.]
 [2. 2.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]

Ground truth:
[[2. 1.]
 [1. 0.]
 [4. 2.]
 [1. 0.]
 [4. 3.]
 [1. 0.]
 [2. 2.]
 [1. 0.]
 [0. 0.]
 [0. 0.]]

Failed:	[46 56]


In [1349]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [1400]:
# Cross Validation Y1
results_train = cross_validate(model5, X, y1,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00697351, 0.00698066, 0.00578094, 0.00501895, 0.00604463]),
 'score_time': array([0.00598431, 0.00299025, 0.00300717, 0.00296092, 0.00296092]),
 'test_f1_scorer': array([-0.7961165 , -0.78640777, -0.87254902, -0.88235294, -0.85294118]),
 'train_f1_scorer': array([-0.85574572, -0.84352078, -0.85121951, -0.84146341, -0.83902439]),
 'test_accuracy': array([0.7961165 , 0.78640777, 0.87254902, 0.88235294, 0.85294118]),
 'train_accuracy': array([0.85574572, 0.84352078, 0.85121951, 0.84146341, 0.83902439])}

In [1402]:
# Cross Validation Y2
results_train = cross_validate(model5, X, y2,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00897336, 0.00698137, 0.0059433 , 0.00495553, 0.0069778 ]),
 'score_time': array([0.00398946, 0.00202584, 0.00298762, 0.00299239, 0.00399399]),
 'test_f1_scorer': array([-0.75728155, -0.81553398, -0.75490196, -0.7745098 , -0.85294118]),
 'train_f1_scorer': array([-0.799511  , -0.80195599, -0.8       , -0.79756098, -0.78780488]),
 'test_accuracy': array([0.75728155, 0.81553398, 0.75490196, 0.7745098 , 0.85294118]),
 'train_accuracy': array([0.799511  , 0.80195599, 0.8       , 0.79756098, 0.78780488])}